In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from time import sleep
import requests as r 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np 
import gspread

In [ ]:


while True:
    try:
        credentials = {
            "type": "service_account",
            "project_id": "spread-sheet-for-fb-marketplae",
            "private_key_id": "Your private id",
            "private_key": "Your private key",
            "client_email": "ss-fb-marketplace@spread-sheet-for-fb-marketplae.iam.gserviceaccount.com",
            "client_id": "your client id",
            "auth_uri": "https://accounts.google.com/o/oauth2/auth",
            "token_uri": "https://oauth2.googleapis.com/token",
            "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
            "client_x509_cert_url": ""
        client = gspread.service_account_from_dict(credentials)
        title = "sheet1"
        sheet_name = "CARS FB MARKETPLACE"
        Worksheet_Reader = client.open(sheet_name).worksheet(title)

        car_dict = {}
        no_of_cars = 1
        option = Options()
        option.add_argument("--disable-infobars")
        option.add_argument("start-maximized")
        option.add_argument("--disable-extensions")

        # Pass the argument 1 to allow and 2 to block
        option.add_experimental_option("prefs", { 
            "profile.default_content_setting_values.notifications": 1 
        })
        driver = webdriver.Chrome(executable_path="C:\chromedriver.exe", chrome_options=option)
        url = "https://www.facebook.com/login/"
        driver.get(url)
        sleep(5)
        # Facebook Credentials
        email_box = driver.find_element_by_id("email")
        email_box.send_keys("Number or Email")
        password_box = driver.find_element_by_id("pass")
        password_box.send_keys("Password")
        password_box.send_keys(Keys.ENTER)
        sleep(3)
        # MORE SCROLL COUNTER MEANS MORE CAR ITEMS
        url = "https://www.facebook.com/marketplace/106270089409734/vehicles/?sortBy=creation_time_descend&exact=false"
        driver.get(url)
        sleep(2)
        scroll_counter = 5
        for i in range(1, scroll_counter):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(5)
        sleep(3)
        data = driver.page_source
        soup = bs(data, "html.parser")
        cars = soup.find_all('div', {
            "class": "b3onmgus ph5uu5jm g5gj957u buofh1pr cbu4d94t rj1gh0hx j83agx80 rq0escxv fnqts5cd fo9g3nie n1dktuyu e5nlhep0 ecm0bbzt"})
        print(len(cars))
        row = 2
        # first value of previous car is nothing since there is no car stored yet
        previous_car = ""
        for car in cars:
            try:
                desc_url = "https://web.facebook.com" + car.find('a').get('href')
                price = car.find('span', {
                    'class': "d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em mdeji52x a5q79mjw g1cxx5fr lrazzd5p oo9gr5id"}).text.strip()[
                        1:]
                year_model = car.find('div', {'class': "a8nywdso e5nlhep0 rz4wbd8a linoseic"}).text.strip()
                year = year_model[:4]
                model = year_model[5:]
                region = car.find_all('span', {'class': "a8c37x1j ni8dbmo4 stjgntxs l9j0dhe7 ltmttdrg g0qnabr5 ojkyduve"})[
                    0].text.strip()
                if "Canberra" in region:
                    continue
                mileage = car.find_all('span', {'class': "a8c37x1j ni8dbmo4 stjgntxs l9j0dhe7 ltmttdrg g0qnabr5 ojkyduve"})[
                    1].text.strip()
                print(model, year, mileage, price, region, desc_url)
            except:
                continue

            # Compares previous car with current car if both are different then store data else skipp
            if year_model != previous_car:
                array = np.array([[model, year, mileage, price, region, desc_url]])
                Worksheet_Reader.update('A' + str(row), array.tolist())
                row += 1
                sleep(1)
            # store the current car year and model into previous variable
            previous_car = year_model
        break
        sleep(60*10)
        print("SCRAPING OF ALL LATEST CARS COMPLETED")
    except:
        print("THERE WAS AN EXCEPTION")
        print("TRYING AGAIN")
        break

